### SparkContext and RDD basics

In [2]:
import findspark
findspark.init()

In [3]:
from pyspark import SparkContext
import numpy as np

### Initialize a SparkContext (the main abstraction to the cluster)

#### Note the '4' in the argument. It denotes 4 cores to be used for this SparkContext object.

In [4]:
sc=SparkContext(master="local[4]")

In [5]:
print(sc)

<SparkContext master=local[4] appName=pyspark-shell>


### Generate a list of random integeres

In [6]:
lst=np.random.randint(0,10,20)

In [7]:
print(lst)

[2 5 4 3 4 1 6 1 4 5 3 9 6 7 7 4 6 8 5 5]


### Parallelize the list - this is the main operation toward distributed computing

In [8]:
A=sc.parallelize(lst)

### What did we just do? We created a RDD? What is a RDD?

<img src="images/sparkalgo.png" />

Spark revolves around the concept of a resilient distributed dataset (RDD), which is a **fault-tolerant collection of elements that can be operated on in parallel**. SparkContext manages the distributed data over the worker nodes through the cluster manager.

There are two ways to create RDDs:

- Parallelizing an existing collection in your driver program, or
- Referencing a dataset in an external storage system, such as a shared filesystem, HDFS, HBase, or any data source offering a Hadoop InputFormat.

We created a RDD using the former approach.

#### A is a pyspark RDD object, we cannot access the elements directly


In [9]:
type(A)

pyspark.rdd.RDD

In [10]:
A

ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:195

#### Opposite to parallelization - collect brings all the distributed elements and returns them to the head node. 

#### Note - this is a slow process, do not use it often.


In [11]:
A.collect()

[2, 5, 4, 3, 4, 1, 6, 1, 4, 5, 3, 9, 6, 7, 7, 4, 6, 8, 5, 5]

#### How were the partitions created? Use glom method

In [12]:
A.glom().collect()

[[2, 5, 4, 3, 4], [1, 6, 1, 4, 5], [3, 9, 6, 7, 7], [4, 6, 8, 5, 5]]

#### Now stop the SC and reinitialize it with 2 cores and see what happens when you repeat the process!

In [13]:
sc.stop()

In [14]:
sc = SparkContext(master="local[2]")
A = sc.parallelize(lst)
A.glom().collect()

[[2, 5, 4, 3, 4, 1, 6, 1, 4, 5], [3, 9, 6, 7, 7, 4, 6, 8, 5, 5]]

#### The RDD is now distributed over two chunks, not four!
#### So, let's redo the process with 4 cores again.

In [15]:
sc.stop()
sc = SparkContext(master="local[4]")
A = sc.parallelize(lst)

### Basic operations

In [17]:
A.count() # Count the elements

20

In [18]:
A.first() # The first element (first)

2

In [19]:
A.take(4) # the first few elements (take)

[2, 5, 4, 3]

#### Removing duplicates: Get another RDD with only the distinct elements

The method RDD.distinct() Returns a new dataset that contains the distinct elements of the source dataset.

**NOTE**: This operation requires a **shuffle** in order to detect duplication across partitions. **So, it is a slow operation.**

In [20]:
A_distinct=A.distinct()
A_distinct.collect()

[4, 8, 5, 1, 9, 2, 6, 3, 7]

#### To sum all the elements use reduce method

In [1]:
A.reduce(lambda x,y:x+y)

NameError: name 'A' is not defined

In [22]:
A.sum() # Direct sum method

95

#### Or using the fold method, which aggregates the elements of each partition, and then the results for all the partitions

In [23]:
A.fold(0,lambda x,y:x+y)

95

In [24]:
A.reduce(lambda x,y: x if x > y else y) # Finding maximum element by reduce

9

In [25]:
# Finding longest word using reduce

words = 'These are some of the best Macintosh computers ever'.split(' ')
wordRDD = sc.parallelize(words)
wordRDD.reduce(lambda w,v: w if len(w)>len(v) else v)

'computers'

### Functions/filtering over RDD

#### Use filter to return a new RDD with elements satisfying a given predicate (lambda expression)

In [26]:
A.filter(lambda x: x%3==0).collect() # # Return RDD with elements divisible by 3

[3, 6, 3, 9, 6, 6]

#### Lambda functions are short and sweet but we can write regular Python functions to use with reduce

In [27]:
def largerThan(x,y):
    """
    Returns the last word among the longest words in a list
    """
    if len(x)> len(y):
        return x
    elif len(y) > len(x):
        return y
    else:
        if x < y: return x
        else: return y
        
wordRDD.reduce(largerThan)

'Macintosh'